In [12]:
import boto3
import os
from decimal import Decimal
from dotenv import load_dotenv
import json
from boto3.dynamodb.conditions import Key

load_dotenv()


aws_access_key_id = os.getenv('AWS_ACCESS_KEY_ID')
aws_secret_access_key = os.getenv('AWS_SECRET_ACCESS_KEY')
region_name = os.getenv('AWS_DEFAULT_REGION')


dynamodb = boto3.resource(
    'dynamodb',
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
    region_name=region_name
)


# Custom class to handle Decimal types
class DecimalEncoder(json.JSONEncoder):
    def default(self, o):
        if isinstance(o, Decimal):
            return float(o)
        return super(DecimalEncoder, self).default(o)

def export_dynamodb_to_json(table_name, output_file):
    # Initialize DynamoDB client
    table = dynamodb.Table(table_name)
    
    # Scan the entire table
    response = table.scan()
    data = response['Items']
    
    # Keep scanning until all items are retrieved
    while 'LastEvaluatedKey' in response:
        print("Fetching more data...")
        response = table.scan(ExclusiveStartKey=response['LastEvaluatedKey'])
        data.extend(response['Items'])
    
    # Save to JSON file
    with open(output_file, 'w') as f:
        json.dump(data, f, cls=DecimalEncoder, indent=4)
    
    print(f"Data exported to {output_file} successfully.")



In [13]:
tbls = [
    "vbr_bot_career_convos",
    "vbr_bot_career_messages",
    "vbr_bot_career_state",
    "vbr_bot_career_emotions",
    "vbr_bot_career_values",
    "vbr_bot_career_reappraisals",
]

for tbl in tbls:
    tbl_name = tbl
    output_file = f"{tbl_name}.json"
    output_file = os.path.join("data", output_file)
    export_dynamodb_to_json(tbl_name, output_file)

Data exported to data/vbr_bot_career_convos.json successfully.
Data exported to data/vbr_bot_career_messages.json successfully.
Data exported to data/vbr_bot_career_state.json successfully.
Data exported to data/vbr_bot_career_emotions.json successfully.
Data exported to data/vbr_bot_career_values.json successfully.
Data exported to data/vbr_bot_career_reappraisals.json successfully.
